In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
df_org = pd.read_csv("./data/DailyGrand/DailyGrand.raw.csv", parse_dates=['Date'])

In [3]:
df = df_org[[f"Draw{i}" for i in range(1, 6)]].iloc[7:]
df_test = df_org[[f"Draw{i}" for i in range(1, 6)]].iloc[1:7]

In [4]:
df_test

,Draw1,Draw2,Draw3,Draw4,Draw5
1,9,26,28,29,48
2,10,24,33,43,44
3,2,3,20,26,28
4,17,32,33,41,45
5,9,39,40,41,45
6,1,26,27,38,46


In [5]:
scaler = StandardScaler().fit(df.values)
ds_transformed = scaler.transform(df.values)
df_transformed = pd.DataFrame(data=ds_transformed, index=df.index)

In [6]:
df_transformed.head()

,0,1,2,3,4
7,-0.325024,-1.028464,-1.607447,-0.175018,-1.428477
8,2.645922,1.811347,1.842385,1.109794,0.599102
9,-0.481390,-1.146790,-1.391832,-0.175018,1.067005
10,0.769535,0.628092,0.656506,0.408988,0.443135
11,-0.950486,-1.620091,-2.146483,-2.511041,-2.364283


In [7]:
number_of_rows = df.shape[0]
window_length = 6
number_of_features = df.shape[1]

In [8]:
train = np.empty([number_of_rows - window_length, window_length, number_of_features], dtype=float)
label = np.empty([number_of_rows - window_length, number_of_features], dtype=float)

In [9]:
for i in range(number_of_rows - window_length):
    train[i] = df_transformed.iloc[i: i+window_length, 0: number_of_features]
    label[i] = df_transformed.iloc[i+window_length: i+window_length+1, 0: number_of_features]

In [10]:
train.shape, label.shape

((630, 6, 5), (630, 5))

## LSTM Mode

In [11]:
from keras import models
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

2022-12-26 22:08:47.106338: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-26 22:08:47.106375: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-26 22:08:48.062015: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-26 22:08:48.062114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-26 22:08:48.062127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [12]:
batch_size = 49

In [13]:
model = Sequential()
model.add(LSTM(240,
                            input_shape=(window_length, number_of_features),
                            return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(240,
                            input_shape=(window_length, number_of_features),
                            return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(240,
                            input_shape=(window_length, number_of_features),
                            return_sequences=True))
model.add(LSTM(240,
                            input_shape=(window_length, number_of_features),
                            return_sequences=False))
model.add(Dense(49))
model.add(Dense(number_of_features))
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])

2022-12-26 22:08:48.901811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-26 22:08:48.901841: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-26 22:08:48.901864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f9c5764e8a97): /proc/driver/nvidia/version does not exist


In [ ]:
model.fit(train, label, batch_size=100, epochs=30)

Epoch 1/30


Exception ignored in: <function UniquePtr.__del__ at 0x7f0ebddb4820>
Traceback (most recent call last):
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/tensorflow/python/framework/c_api_util.py", line 70, in __del__
    def __del__(self):
KeyboardInterrupt: 


In [ ]:
import random

In [ ]:
to_predict = np.array(df_test.values)
scaled_to_predict = scaler.transform(to_predict)
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

In [ ]:
model.summary()

In [ ]:
df_test.values

In [ ]:
df = pd.read_csv('./cold_combine.csv')

In [ ]:
df = df[df['ColdPairs'].str.contains(" - ")].reset_index(drop=True)

In [ ]:
df[['Num1','Num2']] = df['ColdPairs'].str.split(' - ',expand=True)

In [ ]:
df.to_csv('cold_pairs.csv', index=False)